<a href="https://colab.research.google.com/github/rodrigowue/Nutt-TDC/blob/main/notebook_tdc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

# Synthesizable Nutt-Based TDC With Self-Timing Calibration


```
SSCS TC-OSE Team, February 2023
SPDX-License-Identifier: Apache-2.0
```


|Name|Email|Affiliation|IEEE Member|SSCS Member|
|:--:|:--:|:----------:|:----------:|:----------:|
|Rodrigo Nogueira Wuerdig|rnwuerdig@inf.ufrgs.br|UFRGS|Yes|Yes|
|Author 2|Email2|Affiliation 2|Yes/No|Yes/No|
|Author 3|Email3|Affiliation 3|Yes/No|Yes/No|
|Author 4 (Lead) <br />|Email4|Affiliation 4|Yes/No|Yes/No|

**_Abstract_** - This electronic document is a “live” template and already defines the components of your paper [title, text, heads,
etc.] in its style sheet. *

**_Key words_** - component; formatting; style; styling; insert (key words)


## Tool Installation

This is where you need to install your tools. We provide here an example where conda environment is being installed and then Ngspice for simulations.


**_Tool setup adopted from @proppy and @bmurmann (see this [Colab notebook](https://colab.research.google.com/gist/proppy/a0c5ed3e28e942f1621200dcf67bad5a/sky130-pyspice-playground.ipynb#scrollTo=q0XHBAt1jGmQ))_**

In [ ]:
#@title Install dependencies {display-mode: "form"}
#@markdown - Click the ▷ button to setup the digital design environment based on [conda-eda](https://github.com/hdl/conda-eda).

ngspice_version = 'latest' #@param {type:"string"}
gf180mcu_fd_pr_version = 'latest' #@param {type:"string"}

if ngspice_version == 'latest':
  ngspice_version = ''

if gf180mcu_fd_pr_version == 'latest':
  gf180mcu_fd_pr_version = 'main'

import os
import pathlib
import urllib.request

!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!CI=0 bin/micromamba install --yes --prefix $CONDA_PREFIX \
                     --channel litex-hub \
                     --channel main \
                     ngspice={ngspice_version}

ngspice_models_dir = pathlib.Path('globalfoundries-pdk-libs-gf180mcu_fd_pr/models/ngspice')
ngspice_models_dir.mkdir(exist_ok=True, parents=True)
models = ['design.ngspice', 'sm141064.ngspice', 'sm141064_mim.ngspice', 'smbb000149.ngspice']
for m in models:
  url = f'https://github.com/efabless/globalfoundries-pdk-libs-gf180mcu_fd_pr/raw/{gf180mcu_fd_pr_version}/models/ngspice/{m}'
  print('downloading:', url)
  with urllib.request.urlopen(url) as src:
    with (ngspice_models_dir / m).open('wb') as dst:
      dst.write(src.read())

PATH = os.environ['PATH']
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}

## Introduction

A Time-to-Digital circuit is implemented in order to quantify the time displacement between events. As in the figure below, it takes two inputs (start and stop) and quantifies the time displacement ($\Delta t$). This displacement is then presented in form of a digital word output that can be encoded in different ways, e.g., binary, or thermometer code. TDCs can be found in a variety of circuits, especially mixed-signal circuits such as Phase-Locked Loops, SAR ADCs, Particle Detectors, etc.

![tdc_simple](https://drive.google.com/uc?export=view&id=1wf7zMJLH3VdMqPxYrMS9rvrz7VTMNPbE)

Advances in CMOS technology increased unit capacitance, but shrinkage of CMOS gate dimensions reduced transistor total gate capacitance significantly. The reduction in total gate capacitance led to a major reduction in gate delay. Integrated circuits for IoT devices are usually employed using technologies where the mean gate delay for a typical digital transistor sits at a few picoseconds. Even under extreme low-voltage supply conditions, the pessimistic perspective is on the order of a few nanoseconds.

Due to their fast-switching nature, CMOS circuits are not friendly for measuring long periods employing single-shot type TDCs, e.g., Flash TDC and Vernier TDC.

Since most of the previously proposed TDCs were meant to measure very small periods or required at least 1 MHz clock, this document proposes a novel hybrid TDC that can measure asynchronous events while keeping only an external 32.768 kHz clock. The proposed Hybrid Time-to-digital utilizes a coarse single counter TDC, fed with the 32.768 kHz clock. That frequency enables counting in a $30.52\mu s$ granularity. In order to measure variations below that, a interpolation method called Nutt-Interpolation method is used.



## II. Implementation Details of your Idea

![TDC.png](https://drive.google.com/uc?export=view&id=1gUrGSERwZz6JLBstC_j7euhnAtei-kso)




**Writing the Verilog Files**

In [ ]:
%writefile ro.v
`include "/content/conda-env/share/pdk/sky130A/libs.ref/sky130_fd_sc_hd/verilog/sky130_fd_sc_hd.v"
module ro
#(parameter N=29)
(
	input enable_i,
	output freq_o
);
wire[N-1:0] ro;
sky130_fd_sc_hd__nand2 nand2(.Y(ro[0]), .A(enable_i), .B(ro[N-1]));

genvar i;
generate
	for (i=0;i<N-1; i=i+1) begin
			sky130_fd_sc_hd__inv inv0(.Y(ro[i+1]), .A(ro[i]));
	end
endgenerate
assign freq_o = ro[0];
endmodule

## Write configuration

[Documentation](https://openlane.readthedocs.io/en/latest/reference/configuration.html)

In [ ]:
%%writefile config.json
{
    "DESIGN_NAME": "ro",
    "VERILOG_FILES": "dir::ro.v",
    "STD_CELL_LIBRARY": "sky130_fd_sc_hd",
    "RSZ_DONT_TOUCH_RX": "ro*",
    "PL_RANDOM_GLB_PLACEMENT": true,
    "FP_SIZING": "absolute",
    "DIE_AREA": "0 0 50 50",
    "DESIGN_IS_CORE": 0,
    "PL_TARGET_DENSITY": 0.70,
    "FP_PDN_AUTO_ADJUST": false,
    "FP_PDN_VPITCH": 25,
    "FP_PDN_HPITCH": 25,
    "FP_PDN_VOFFSET": 5,
    "FP_PDN_HOFFSET": 5,
    "DIODE_INSERTION_STRATEGY": 3
}

## Run OpenLane Flow

[OpenLane](https://openlane.readthedocs.io/en/latest/) is an automated [RTL](https://en.wikipedia.org/wiki/Register-transfer_level) to [GDSII](https://en.wikipedia.org/wiki/GDSII) flow based on several components including [OpenROAD](https://theopenroadproject.org/), [Yosys](https://yosyshq.net/yosys/), [Magic](http://www.opencircuitdesign.com/magic/), [Netgen](http://opencircuitdesign.com/netgen/) and custom methodology scripts for design exploration and optimization targeting [open source PDKs](https://github.com/google/open-source-pdks).


In [ ]:
%env PDK=sky130A
!flow.tcl -design .

## Display layout

In [ ]:
import pathlib
import gdstk
import IPython.display

gdss = sorted(pathlib.Path('runs').glob('*/results/final/gds/*.gds'))
library = gdstk.read_gds(gdss[-1])
top_cells = library.top_level()
top_cells[0].write_svg('inverter.svg')
IPython.display.SVG('inverter.svg')

## Metrics

[Documentation](https://openlane.readthedocs.io/en/latest/reference/datapoint_definitions.html)


In [ ]:
import pandas as pd
import pathlib

pd.options.display.max_rows = None
final_summary_reports = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(final_summary_reports[-1])
df.transpose()

## III. Summary of your Idea

This is where you summarize your work. Comparison tables and a description of your expected results should be listed here.

## IV. Planification and Tasks

Planning and breakdown of tasks are usually helpful for a successful project. Good Luck!